# Inicialização do Código

In [66]:
# criar um navegador
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
import pandas as pd
import openpyxl
import time
import smtplib



nav = webdriver.Chrome()

# importar/visualizar a base de dados
tabela_produtos = pd.read_excel('buscas.xlsx')
display(tabela_produtos)

,Nome,Termos banidos,Preço mínimo,Preço máximo
0,iphone 12 64gb,mini watch,3000,3500
1,rtx 3060,zota galax,4000,4500


# Definição de Funções

In [67]:
def verificar_tem_termos_banidos(lista_termos_banidos, nome):
    tem_termos_banidos = False
    for palavra in lista_termos_banidos:
            if palavra in nome.lower():
                tem_termos_banidos = True
    return tem_termos_banidos

def verificar_tem_todos_termos_produto(lista_termos_nome_produto, nome):
    tem_todos_termos_produtos = True
    for palavra in lista_termos_nome_produto:
        if palavra not in nome.lower():
            tem_todos_termos_produtos = False
    return tem_todos_termos_produtos

In [68]:
# pesquisar pelo produto

        


# para cada item dentro da nossa base de dados, para cada produto
    # procurar esse produto no google shopping
        # verificar se algum dos produtos do google shopping está dentro da minha faixa de preço
    # procurar esse produto no buscapé
        # verificar se algum dos produtos do buscapé está dentro da minha faixa de preço

# salvar as ofertas boas em um dataframe (tabela)
# exportar pro excel
# enviar por email o resultado da tabela

In [69]:
def busca_google_shopping(nav, produto, termos_banidos, preco_min, preco_max):
    produto = produto.lower()
    termos_banidos = termos_banidos.lower()
    lista_termos_nome_produto = produto.split()
    lista_termos_banidos = termos_banidos.split()
    lista_ofertas = []
    preco_min = float(preco_min)
    preco_max = float(preco_max)

    # abrir google
    nav.get('https://www.google.com/')
    nav.find_element(By.XPATH, '//*[@id="APjFqb"]').send_keys(produto, Keys.ENTER) # primeiro escreve o produto e depois aperta enter
    # entrar na aba shopping
    nav.find_element(By.XPATH, '//*[@id="hdtb-sc"]/div/div/div[1]/div/div[2]/a').click()
    # é bom por aspas simples quando copiar xpath, pois o xpath pode ter aspas duplas

    # pegar as informações do produto
    lista_resultados = nav.find_elements(By.CLASS_NAME, 'i0X6df')
    for resultado in lista_resultados:  
        nome = resultado.find_element(By.CLASS_NAME, 'tAxDx').text
        # analisar se não tem nenhum termo banido
       
        if not verificar_tem_termos_banidos(lista_termos_banidos, nome) and verificar_tem_todos_termos_produto(lista_termos_nome_produto, nome):
            preco = resultado.find_element(By.CLASS_NAME, 'a8Pemb').text
            preco = preco.replace('R$', '').replace(' ', '').replace('.', '').replace(',', '.')
            # tirar o ' + impostos'
            if 'impostos' in preco:
                preco = preco.split('+impostos')[0]
            preco = float(preco)

            # verificar se o preco ta entre preco min e max
            if preco_min <= preco <= preco_max:
            
                elemento_ref = resultado.find_element(By.CLASS_NAME, 
                'EI11Pd')
                elemento_pai = elemento_ref.find_element(By.XPATH, '..')
                link = elemento_pai.get_attribute('href')
                #link = resultado.find_element(By.CLASS_NAME, 'Lq5OHe').get_attribute('href')
                lista_ofertas.append((nome, preco, link))
    return lista_ofertas

def busca_buscape(nav, produto, termos_banidos, preco_min, preco_max):
    # tratar os valores
    produto = produto.lower()
    termos_banidos = termos_banidos.lower()
    lista_termos_nome_produto = produto.split()
    lista_termos_banidos = termos_banidos.split()
    lista_ofertas = []
    preco_min = float(preco_min)
    preco_max = float(preco_max)

    # buscar no buscape
    nav.get('https://www.buscape.com.br/')
    nav.find_element(By.XPATH, '//*[@id="new-header"]/div[1]/div/div/div[3]/div/div/div[2]/div/div[1]/input').send_keys(produto, Keys.ENTER)

    # pegar os resultados
    time.sleep(3) # para esperar carregar a página
    lista_resultados = nav.find_elements(By.CLASS_NAME, 'ProductCard_ProductCard_Inner__gapsh')

    for resultado in lista_resultados:
        preco = resultado.find_element(By.CLASS_NAME, 'Text_MobileHeadingS__HEz7L').text
        preco = preco.replace('R$', '').replace(' ', '').replace('.', '').replace(',', '.')
        preco = float(preco)
        nome = resultado.find_element(By.CLASS_NAME, 'ProductCard_ProductCard_Name__U_mUQ').text
        link = resultado.get_attribute('href')
        
    # analisar se o resultado tem termo banido e tem todos os termos do produto
        if not verificar_tem_termos_banidos(lista_termos_banidos, nome) and verificar_tem_todos_termos_produto(lista_termos_nome_produto, nome):
            # verificar se o preco ta entre preco min e max
            if preco_min <= preco <= preco_max:
                lista_ofertas.append((nome, preco, link))
    return lista_ofertas

   

    # analisar se o preco esta entre o preco minimo e o preco maximo

    # retornar a lista de ofertas do buscape



# Construindo a nossa lista de ofertas

In [70]:
tabela_ofertas = pd.DataFrame()
for linha in tabela_produtos.index:
    produto = tabela_produtos.loc[linha, "Nome"]
    termos_banidos = tabela_produtos.loc[linha, "Termos banidos"]
    preco_min = tabela_produtos.loc[linha, "Preço mínimo"]
    preco_max = tabela_produtos.loc[linha, "Preço máximo"]


    lista_ofertas_google_shopping = busca_google_shopping(nav, produto, termos_banidos, preco_min, preco_max)
    if lista_ofertas_google_shopping:
        tabela_google_shopping = pd.DataFrame(lista_ofertas_google_shopping, columns=['Produto', 'Preço', 'Link'])
        tabela_ofertas = pd.concat([tabela_ofertas, tabela_google_shopping])

    lista_ofertas_buscape = busca_buscape(nav, produto, termos_banidos, preco_min, preco_max)
    if lista_ofertas_buscape:
        tabela_buscape = pd.DataFrame(lista_ofertas_buscape, columns=['Produto', 'Preço', 'Link'])
        tabela_ofertas = pd.concat([tabela_ofertas, tabela_buscape])
    


    


# Exportando para o Excel

In [71]:
tabela_ofertas.to_excel("Ofertas.xlsx", index=False)

# Enviando o E-mail

In [72]:
def enviar_email():  
    texto = f"""
    Prezados,
    Encontramos alguns produtos em oferta dentro da faixa de preço desejada
    {tabela_ofertas.to_string(index=False)}
    Att,
    Pedro
    """

    html = f"""
    <p>Prezados,</p>
    <p>Encontramos alguns produtos em oferta dentro da faixa de preço desejada</p>
    {tabela_ofertas.to_html(index=False)}
    <p>Att,</p>
    <p>Pedro</p>
    """

    msg = MIMEMultipart('alternative')
    msg['Subject'] = "Lista de Ofertas"
    msg['From'] = 'pedrocacoal2013@gmail.com'
    msg['To'] = 'pedro.oliveira6@sou.ufmt.br'
    password = 'rvwi zduq yylb kbnz' 
    msg.add_header('Lista de Ofertas', 'text/html')
    
    parte1 = MIMEText(texto, 'plain')
    parte2 = MIMEText(html, 'html')
    msg.attach(parte1)
    msg.attach(parte2)

    s = smtplib.SMTP('smtp.gmail.com: 587')
    s.starttls()
    # Login Credentials for sending the mail
    s.login(msg['From'], password)
    s.sendmail(msg['From'], [msg['To']], msg.as_string().encode('utf-8'))
    print('Email enviado')
    
nav.quit()

In [73]:
enviar_email()

Email enviado
